# Step Two: Model Checking

## Preparation

In [1]:
# Import stormvogel and prism models from the previous step
import stormvogel
import stormpy

from examples.orchard_builder import build_simple, build_full, build_prism

In [2]:
orchard_simple = build_simple()
orchard, orchard_storm = build_full()
orchard_prism = build_prism()

## Reachability

In [3]:
# Probability of winning the game
prob_players_won = 'Pmax=? [F "PlayersWon"]'
result = stormvogel.model_checking(orchard_simple, prob_players_won)
print(result.get_result_of_state(orchard_simple.get_initial_state()))

WARN  (IterativeMinMaxLinearEquationSolver.cpp:191): Expected VI operator to be initialized for scheduler extraction. Initializing now, but this is inefficient.
WARN  (IterativeMinMaxLinearEquationSolver.cpp:191): Expected VI operator to be initialized for scheduler extraction. Initializing now, but this is inefficient.
WARN  (IterativeMinMaxLinearEquationSolver.cpp:191): Expected VI operator to be initialized for scheduler extraction. Initializing now, but this is inefficient.
WARN  (IterativeMinMaxLinearEquationSolver.cpp:191): Expected VI operator to be initialized for scheduler extraction. Initializing now, but this is inefficient.
WARN  (IterativeMinMaxLinearEquationSolver.cpp:191): Expected VI operator to be initialized for scheduler extraction. Initializing now, but this is inefficient.
WARN  (IterativeMinMaxLinearEquationSolver.cpp:191): Expected VI operator to be initialized for scheduler extraction. Initializing now, but this is inefficient.
WARN  (IterativeMinMaxLinearEquati

In [ ]:
vis = stormvogel.show(orchard_simple, result=result)

## Total rewards

In [ ]:
reward_prop = 'R{"rounds"}max=? [F "PlayersWon" | "RavenWon"]'
print(stormvogel.model_checking(orchard, reward_prop).get_result_of_state(orchard.get_initial_state()))
reward_prop = 'R{"rounds"}min=? [F "PlayersWon" | "RavenWon"]'
print(stormvogel.model_checking(orchard, reward_prop).get_result_of_state(orchard.get_initial_state()))

## Beyond

In [5]:
# Helper function
def model_check(model, prop):
    formula = stormpy.parse_properties(prop)[0]
    result = stormpy.model_checking(model, formula, only_initial_states=True)
    return result.at(model.initial_states[0])

### Reward-bounded reachability probabilities 

In [ ]:
# Winning probability within k rounds
probabilities = []
for k in range(41):
    win_steps = 'Pmax=? [F{"rounds"}<=' + str(k) + ' "PlayersWon"]'
    probabilities.append(model_check(orchard_prism, win_steps))
    print("Round {}: {}".format(k, probabilities[-1]))

In [ ]:
# Plot
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.xlabel('Number of rounds')
plt.ylabel('Probability of players winning')
plt.title("Orchard")

# Plot results for all steps
plt.plot(range(len(probabilities)), probabilities, linewidth=2);

### Conditional reachability probabilities 

In [7]:
print(model_check(orchard_prism, 'Pmax=? [F "PlayersWon" || F "RavenOneAway"]'))

0.31979517678214064


### Multi-objective queries

In [ ]:
query = 'multi(R{"rounds"}max=? [F ("PlayersWon" | "RavenWon")], P>=0.60 [F "PlayersWon"])'
print(model_check(orchard_prism, query))